<div class="alert alert-block alert-info text-center">
    <H1> PHASE I (cont.) - preparing our data </H1>
</div>


## Getting all dataframes ready to explore

- Data In & Data Out
- Getting the price history for each ticker
- Getting business information to enrich our analysis
- Understanding the big picture

In [71]:
#imports

import pandas as pd
from glob import glob
from time import strftime, sleep
import numpy as np
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay
import yfinance as yf
yf.pdr_override()
from cryptocmd import CmcScraper

import requests
from dotenv import load_dotenv
import os


def clean_header(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '', regex=False).str.replace('(', \
                '', regex=False).str.replace(')', '', regex=False).str.replace(' ', '_', regex=False).str.replace('_/_', '/', regex=False)
    
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

def datetime_maker(df, datecol):
    df[datecol] = pd.to_datetime(df[datecol])
    

#### Import the last transactions_finaldf from Phase I

In [72]:
# Read excel file transactions_finaldf

last_file = sorted(glob('../outputs/transactions_all/transactions_finaldf_2023-04-14_20h38m.xlsx'))[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
all_transactions = pd.read_excel(last_file, engine='openpyxl')
all_transactions.date = pd.to_datetime(all_transactions.date, format='%d/%m/%Y')


transactions_finaldf_2023-04-14_20h38m.xlsx


In [73]:

all_tickers = list(all_transactions['ticker'].unique())
print('You have {} different Assets'.format(len(all_tickers)))


You have 11 different Assets


In [74]:
all_tickers

['CRO',
 'ADA',
 'BTC',
 'USDM',
 'VET',
 'USDC',
 'ONE',
 'DOGE',
 'AUDIO',
 'HBAR',
 'USD']

### Collecting the price history for all tickers

- You can define the start date for the history below
- Datareader will get each stock individually
- all_data will have all the prices for every ticker

In [75]:
ly = datetime.today().year-1
today = datetime.today()
start_sp = datetime(2020, 1, 1)
end_sp = today
start_asset = datetime(2020, 1, 1)
end_asset = today
start_ytd = datetime(ly, 12, 31) + BDay(1) # to get the first business day

#create a function to fetch histgorical data for all tickers in all_tickers
scraper = CmcScraper('name')
def get(Symbol, startdate, enddate):
    def data(ticker):
        return (scraper.get_dataframe(ticker, start=startdate, end=enddate))
    datas = map(data, Symbol)
    return(pd.concat(datas, keys=Symbol, names=['ticker', 'date']))
               
all_data = get(all_tickers, start_asset, end_asset)

In [76]:
clean_header(all_data)
# all_data = all_data.drop(columns=["market_cap"])

In [77]:
all_data

open      high       low     close      volume  \
ticker date                                                             
CRO    2022-11-24  0.039939  0.039939  0.039939  0.039939        0.00   
       2022-11-23  0.039939  0.039939  0.039939  0.039939        0.00   
       2022-11-22  0.039939  0.039939  0.039939  0.039939        0.00   
       2022-11-21  0.039939  0.039939  0.039939  0.039939        0.00   
       2022-11-20  0.039939  0.039939  0.039939  0.039939        0.00   
...                     ...       ...       ...       ...         ...   
USD    2021-04-20  3.211668  3.226304  1.359353  1.780480  1135265.84   
       2021-04-19  2.730713  3.781825  2.425066  3.215858  1594253.19   
       2021-04-18  3.324406  3.702015  2.267315  2.728804   506215.10   
       2021-04-17  2.802972  4.303353  2.395653  3.327406  1819516.09   
       2021-04-16  5.576011  5.576011  2.421233  2.801766  7002494.86   

                   market_cap  
ticker date                    
CRO    2022-11-24   125294.27  
       2022-11-23   125294.27  
       2022-11-22   125294.27  
       2022-11-21   125294.27  
       2022-11-20   125294.27  
...                       ...  
USD    2021-04-20        0.00  
       2021-04-19        0.00  
       2021-04-18        0.00  
       2021-04-17        0.00  
       2021-04-16        0.00  

[6468 rows x 6 columns]

In [78]:
all_data['adj_close']= all_data['close']

In [79]:
all_data.rename(columns = {'market_cap':'mktvalue'}, inplace = True)
all_data

open      high       low     close      volume  \
ticker date                                                             
CRO    2022-11-24  0.039939  0.039939  0.039939  0.039939        0.00   
       2022-11-23  0.039939  0.039939  0.039939  0.039939        0.00   
       2022-11-22  0.039939  0.039939  0.039939  0.039939        0.00   
       2022-11-21  0.039939  0.039939  0.039939  0.039939        0.00   
       2022-11-20  0.039939  0.039939  0.039939  0.039939        0.00   
...                     ...       ...       ...       ...         ...   
USD    2021-04-20  3.211668  3.226304  1.359353  1.780480  1135265.84   
       2021-04-19  2.730713  3.781825  2.425066  3.215858  1594253.19   
       2021-04-18  3.324406  3.702015  2.267315  2.728804   506215.10   
       2021-04-17  2.802972  4.303353  2.395653  3.327406  1819516.09   
       2021-04-16  5.576011  5.576011  2.421233  2.801766  7002494.86   

                    mktvalue  adj_close  
ticker date                              
CRO    2022-11-24  125294.27   0.039939  
       2022-11-23  125294.27   0.039939  
       2022-11-22  125294.27   0.039939  
       2022-11-21  125294.27   0.039939  
       2022-11-20  125294.27   0.039939  
...                      ...        ...  
USD    2021-04-20       0.00   1.780480  
       2021-04-19       0.00   3.215858  
       2021-04-18       0.00   2.728804  
       2021-04-17       0.00   3.327406  
       2021-04-16       0.00   2.801766  

[6468 rows x 7 columns]

### Saving the price history

<div class="alert alert-block alert-danger">
<b>Please Note:</b><br>If an item does not have price history, we have to remove it from the portfolio.
    <br>This may happen if the stock gets delisted.
</div>

In [80]:


#We catch the tickers in a list
blacklist = []
for tick in all_tickers:
    try:
        all_data.loc[tick].to_csv('../outputs/price_hist/{}_price_hist.csv'.format(tick))
    except KeyError:
        blacklist.append(tick)
        print(f'Ticker {tick} has no price history to save.')
        pass
blacklist



[]

In [81]:
filt_tickers = [tick for tick in all_tickers if tick not in blacklist]

In [82]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6468 entries, ('CRO', Timestamp('2022-11-24 00:00:00')) to ('USD', Timestamp('2021-04-16 00:00:00'))
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       6468 non-null   float64
 1   high       6468 non-null   float64
 2   low        6468 non-null   float64
 3   close      6468 non-null   float64
 4   volume     6468 non-null   float64
 5   mktvalue   6468 non-null   float64
 6   adj_close  6468 non-null   float64
dtypes: float64(7)
memory usage: 377.7+ KB


<div class="alert alert-block alert-warning">
<b>MEGA datasets:</b><br>These will be handy later when we need to quickly access specific assets.
    <br>We use the all_data dataframe, along with the transactions, to create a day-by-day view of each stock value.
</div>

#### MEGA_DICT is dictionary with all the tickers as keys, and their ticker prices as DF
#### MEGA_DF is a dataframe with all the DF's from MEGA_DICT, concatenated along the columns. Can use filter to select columns

In [83]:
MEGA_DICT = {}  # you have to create it first
min_date = '2020-01-01'  # optional
TX_COLUMNS = ['date','ticker', 'cashflow', 'cml_units', 'cml_cost', 'gain_loss']
tx_filt = all_transactions[TX_COLUMNS]  # keeping just the most relevant ones for now

for ticker in filt_tickers:
    prices_df = all_data[all_data.index.get_level_values('ticker').isin([ticker])].reset_index()
    ## Can add more columns like volume!
    PX_COLS = ['date', 'adj_close'] #'volume', 'adj_close']
    prices_df = prices_df[prices_df.date >= min_date][PX_COLS].set_index(['date'])

    # Making sure we get sameday transactions
    tx_df = tx_filt[tx_filt.ticker==ticker].groupby('date').agg({'cashflow': 'sum',
                                                                 'cml_units': 'last',
                                                                 'cml_cost': 'last',
                                                                 'gain_loss': 'sum'})

   
    # Merging price history and transactions dataframe
    tx_and_prices = pd.merge(prices_df, tx_df, how='outer', left_index=True, right_index=True).fillna('-')
    
    # # Convert the adj_close column to a float
    tx_and_prices['adj_close'] = pd.to_numeric(tx_and_prices['adj_close'], errors='coerce')
    
    # This is to fill the days that were not in our transaction dataframe
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace='-', method='ffill')
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace='-', method='ffill')
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace='-', method='ffill')
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace='-', value=0)
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace='-', value=0)
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace='-', value=0)
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].replace(to_replace='-', value=0)
    # Cumulative sum for the cashflow
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].cumsum()
    tx_and_prices[['cml_cost', 'cml_units']] = tx_and_prices[['cml_cost', 'cml_units']].apply(pd.to_numeric)
    tx_and_prices['avg_price'] = round(tx_and_prices['cml_cost']/tx_and_prices['cml_units'],3)
    tx_and_prices['mktvalue'] = round(tx_and_prices['cml_units']*tx_and_prices['adj_close'],3)
    tx_and_prices = tx_and_prices.add_prefix(ticker+'_')
    # Once we're happy with the dataframe, add it to the dictionary
    MEGA_DICT[ticker] = tx_and_prices

    

In [84]:
MEGA_DICT["VET"].tail(10)

,VET_adj_close,VET_cashflow,VET_cml_units,VET_cml_cost,VET_gain_loss,VET_avg_price,VET_mktvalue
date,,,,,,,
2022-11-15,0.041987,-1107818.4,2100.0,105693.0,0.0,50.33,88.173
2022-11-16,0.040470,-1107818.4,2100.0,105693.0,0.0,50.33,84.987
2022-11-17,0.039934,-1107818.4,2100.0,105693.0,0.0,50.33,83.861
2022-11-18,0.039939,-1107818.4,2100.0,105693.0,0.0,50.33,83.872
2022-11-19,0.039939,-1107818.4,2100.0,105693.0,0.0,50.33,83.872
2022-11-20,0.039939,-1107818.4,2100.0,105693.0,0.0,50.33,83.872
2022-11-21,0.039939,-1107818.4,2100.0,105693.0,0.0,50.33,83.872
2022-11-22,0.039939,-1107818.4,2100.0,105693.0,0.0,50.33,83.872
2022-11-23,0.039939,-1107818.4,2100.0,105693.0,0.0,50.33,83.872


In [85]:
MEGA_DF = pd.concat(MEGA_DICT.values(), axis=1)
MEGA_DF.to_csv('../outputs/mega/MEGA_DF_{}.csv'.format(get_now()))
MEGA_DF.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 601 entries, 2021-03-04 to 2023-02-25
Data columns (total 77 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CRO_adj_close    588 non-null    float64
 1   CRO_cashflow     591 non-null    float64
 2   CRO_cml_units    591 non-null    float64
 3   CRO_cml_cost     591 non-null    float64
 4   CRO_gain_loss    591 non-null    object 
 5   CRO_avg_price    2 non-null      float64
 6   CRO_mktvalue     588 non-null    float64
 7   ADA_adj_close    588 non-null    float64
 8   ADA_cashflow     592 non-null    float64
 9   ADA_cml_units    592 non-null    float64
 10  ADA_cml_cost     592 non-null    float64
 11  ADA_gain_loss    592 non-null    object 
 12  ADA_avg_price    3 non-null      float64
 13  ADA_mktvalue     588 non-null    float64
 14  BTC_adj_close    588 non-null    float64
 15  BTC_cashflow     589 non-null    float64
 16  BTC_cml_units    589 non-null    float64
 1

In [86]:
last_file = sorted(glob('../outputs/mega/MEGA*.csv'))[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
MEGA_DF = pd.read_csv(last_file)

MEGA_DF['date'] = pd.to_datetime(MEGA_DF['date'])
MEGA_DF.set_index('date', inplace=True)

MEGA_DF_2023-04-14_23h19m.csv


In [87]:
#MEGA_DF.filter(regex='mktvalue').fillna(0)



## Creating the daily snapshots of our portfolio

- Taking the MEGA_DF dataframe, we keep just the "market value" column for each stock
- If we sum the rows, it will show us how much our portfolio was worth on that day
- Adding SP500 for reference and calculating some metrics
- saving the portf_allvalues dataframe as csv

In [88]:
portf_allvalues = MEGA_DF.filter(regex='mktvalue').fillna(0)
portf_allvalues['portf_value'] = portf_allvalues.sum(axis=1)
portf_allvalues['portf_value']


date
2021-03-04    0.0
2021-03-07    0.0
2021-03-10    0.0
2021-03-12    0.0
2021-04-13    0.0
             ... 
2023-01-14    0.0
2023-01-20    0.0
2023-01-28    0.0
2023-02-11    0.0
2023-02-25    0.0
Name: portf_value, Length: 601, dtype: float64

In [89]:
# portf_allvalues

In [90]:
# For the S&P500 price return
# You can use other symbols. Look it up on yahoo finance
sp500 = pdr.get_data_yahoo('^GSPC', start_asset, end_sp)
clean_header(sp500)

[*********************100%***********************]  1 of 1 completed


In [91]:
sp500

,open,high,low,close,adj_close,volume
Date,,,,,,
2020-01-02,3244.669922,3258.139893,3235.530029,3257.850098,3257.850098,3459930000
2020-01-03,3226.360107,3246.149902,3222.340088,3234.850098,3234.850098,3484700000
2020-01-06,3217.550049,3246.840088,3214.639893,3246.280029,3246.280029,3702460000
2020-01-07,3241.860107,3244.909912,3232.429932,3237.179932,3237.179932,3435910000
2020-01-08,3238.590088,3267.070068,3236.669922,3253.050049,3253.050049,3726840000
...,...,...,...,...,...,...
2023-04-10,4085.199951,4109.500000,4072.550049,4109.109863,4109.109863,3423650000
2023-04-11,4110.290039,4124.259766,4102.609863,4108.939941,4108.939941,3665830000
2023-04-12,4121.720215,4134.370117,4086.939941,4091.949951,4091.949951,3633120000


In [92]:
# portf_allvalues

In [93]:

#getting the pct change
portf_allvalues = portf_allvalues.join(sp500['adj_close'], how='inner')
portf_allvalues.rename(columns={'adj_close': 'sp500_mktvalue'}, inplace=True)
portf_allvalues['ptf_value_pctch'] = (portf_allvalues['portf_value'].pct_change()*100).round(2)
portf_allvalues['sp500_pctch'] = (portf_allvalues['sp500_mktvalue'].pct_change()*100).round(2)
portf_allvalues['ptf_value_diff'] = (portf_allvalues['portf_value'].diff()).round(2)
portf_allvalues['sp500_diff'] = (portf_allvalues['sp500_mktvalue'].diff()).round(2)
portf_allvalues.rename(columns={'index': 'index_date_new'}, inplace=True)
portf_allvalues.reset_index(inplace=True)
portf_allvalues.rename(columns={'index_date_new': 'index_date'}, inplace=True)
portf_allvalues.rename(columns={'index': 'date'}, inplace=True)
portf_allvalues


,date,CRO_mktvalue,ADA_mktvalue,BTC_mktvalue,USDM_mktvalue,VET_mktvalue,USDC_mktvalue,ONE_mktvalue,DOGE_mktvalue,AUDIO_mktvalue,HBAR_mktvalue,USD_mktvalue,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff
0,2021-03-04,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,3768.469971,NaN,NaN,NaN,NaN
1,2021-03-10,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,3898.810059,NaN,3.46,0.00,130.34
2,2021-03-12,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,3943.340088,NaN,1.14,0.00,44.53
3,2021-04-13,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,4141.589844,NaN,5.03,0.00,198.25
4,2021-04-16,0.0,0.0,0.0,0.0,67382.464,0.0,0.000,0.0,0.0,0.000,0.000,67382.464,4185.470215,inf,1.06,67382.46,43.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,2022-11-18,0.0,0.0,0.0,0.0,83.872,0.0,68.376,0.0,0.0,9.985,5.392,167.625,3965.340088,0.01,0.48,0.02,18.78
408,2022-11-21,0.0,0.0,0.0,0.0,83.872,0.0,68.376,0.0,0.0,9.985,4.793,167.026,3949.939941,-0.36,-0.39,-0.60,-15.40
409,2022-11-22,0.0,0.0,0.0,0.0,83.872,0.0,68.376,0.0,0.0,9.985,4.793,167.026,4003.580078,0.00,1.36,0.00,53.64
410,2022-11-23,0.0,0.0,0.0,0.0,83.872,0.0,68.376,0.0,0.0,9.985,4.793,167.026,4027.260010,0.00,0.59,0.00,23.68


In [94]:

portf_allvalues.to_csv('../outputs/portfolio_df/portfolio_df_{}.csv'.format(get_now()), index=False)


## Getting Sector and Industry
### The "give me everything" method

- In order for us to get a broader view on our portfolio, we want to add the sector and industry to the dataframe


In [95]:
load_dotenv()
API_KEY = os.getenv("CMC_API_KEY")

In [96]:
def get_crypto_data(tickers):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/info"
    parameters = {
        "symbol": ",".join(tickers),
        "CMC_PRO_API_KEY": API_KEY
    }
    response = requests.get(url, params=parameters)
    data = response.json()
    crypto_data = []
    for ticker, info in data['data'].items():
        crypto_data.append([ticker, info['tags'][0]])
    return crypto_data

tickers = ['CRO', 'ADA', 'BTC', 'ONE', 'VET', 'USDC', 'DOGE', 'AUDIO']
crypto_data = get_crypto_data(tickers)
screener_all = pd.DataFrame(crypto_data, columns=["ticker", "sector"])


In [97]:
screener_all

,ticker,sector
0,ADA,dpos
1,AUDIO,music
2,BTC,mineable
3,CRO,medium-of-exchange
4,DOGE,mineable
5,ONE,platform
6,USDC,medium-of-exchange
7,VET,logistics


In [98]:
import requests

def get_cmc_data(ticker):
    headers = {
        "Accepts": "application/json",
        "X-CMC_Pro_API_Key": API_KEY
    }

    url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest?symbol={ticker}&convert=USD"

    response = requests.get(url, headers=headers)
    data = response.json()
    quote = data["data"].get(ticker, {}).get("quote", {}).get("USD", {})

    return {
        "ticker": ticker,
        "price": quote.get("price"),
    }

tickers = ['CRO', 'ADA', 'BTC', 'ONE', 'VET', 'USDC', 'DOGE', 'AUDIO']
data = [get_cmc_data(ticker) for ticker in tickers]
p_1 = pd.DataFrame(data)
p_1


,ticker,price
0,CRO,0.070200
1,ADA,0.440747
2,BTC,30426.898925
3,ONE,0.022616
4,VET,0.025521
5,USDC,1.000041
6,DOGE,0.088995
7,AUDIO,0.347409


In [99]:
screener_all_cl=pd.merge(screener_all, p_1, left_on='ticker', right_on='ticker', how='outer')
screener_all_cl

,ticker,sector,price
0,ADA,dpos,0.440747
1,AUDIO,music,0.347409
2,BTC,mineable,30426.898925
3,CRO,medium-of-exchange,0.070200
4,DOGE,mineable,0.088995
5,ONE,platform,0.022616
6,USDC,medium-of-exchange,1.000041
7,VET,logistics,0.025521


# Last Positions

## Getting the latest values

- In order to get the latest position value, we need to get the latest prices from yahoo finance
- Since we also want sector and industry to be able to segment our portfolio, we will get data from finviz too


In [102]:
last_positions = all_transactions.groupby(['ticker']).agg({'cml_units': 'last', 'cml_cost': 'last',
                                                'gain_loss': 'sum', 'cashflow': 'sum'}).reset_index()
last_positions.head(11)

,ticker,cml_units,cml_cost,gain_loss,cashflow
0,ADA,0.0,0.00,0.00,-32554.10
1,AUDIO,0.0,0.00,-2111.50,-5356.13
2,BTC,0.0,0.00,0.00,0.00
3,CRO,0.0,0.00,0.00,-38306.00
4,DOGE,0.0,0.00,-62.26,-22206.31
5,HBAR,250.0,3285.00,0.00,-37332.50
6,ONE,946.0,14672.46,0.00,-109907.30
7,USD,15.0,225.00,0.00,-2475.00
8,USDC,0.0,0.00,-9767.84,-11950.60
9,USDM,0.0,0.00,0.00,-4222.40


In [104]:
last_positions['price'] = screener_all_cl['price']
last_positions['current_value'] = (last_positions.price * last_positions.cml_units).round(2)
last_positions['avg_price'] = (last_positions.cml_cost / last_positions.cml_units).round(2)
last_positions = last_positions.sort_values(by='current_value', ascending=False)

last_positions.head(11)

,ticker,cml_units,cml_cost,gain_loss,cashflow,price,current_value,avg_price
6,ONE,946.0,14672.46,0.00,-109907.30,1.000041,946.04,15.51
5,HBAR,250.0,3285.00,0.00,-37332.50,0.022616,5.65,13.14
7,USD,15.0,225.00,0.00,-2475.00,0.025521,0.38,15.00
0,ADA,0.0,0.00,0.00,-32554.10,0.440747,0.00,NaN
1,AUDIO,0.0,0.00,-2111.50,-5356.13,0.347409,0.00,NaN
2,BTC,0.0,0.00,0.00,0.00,30426.898925,0.00,NaN
3,CRO,0.0,0.00,0.00,-38306.00,0.070200,0.00,NaN
4,DOGE,0.0,0.00,-62.26,-22206.31,0.088995,0.00,NaN
8,USDC,0.0,0.00,-9767.84,-11950.60,NaN,NaN,NaN
9,USDM,0.0,0.00,0.00,-4222.40,NaN,NaN,NaN


## Final Last Positions

<div class="alert alert-block alert-success">
<b>Finally!</b><br> This last dataframe will give you everything you need to build an awesome dashboard.
</div>

In [105]:
# final_lastpositions = pd.merge(finviz_merged_clean, last_positions, left_on='ticker', right_on='ticker', how='outer')
final_lastpositions = pd.merge(screener_all, last_positions, left_on='ticker', right_on='ticker', how='outer')
final_lastpositions['current_value'] = final_lastpositions.price*final_lastpositions.cml_units
final_lastpositions['avg_price'] = final_lastpositions.cml_cost/final_lastpositions.cml_units
final_lastpositions['portf_weight'] = final_lastpositions.current_value/final_lastpositions.current_value.sum()*100
final_lastpositions['unrealizedval'] = final_lastpositions.current_value - final_lastpositions.cml_cost
final_lastpositions['unrealizedpct'] = final_lastpositions.unrealizedval / final_lastpositions.cml_cost*100
final_lastpositions.round(2)

,ticker,sector,cml_units,cml_cost,gain_loss,cashflow,price,current_value,avg_price,portf_weight,unrealizedval,unrealizedpct
0,ADA,dpos,0.0,0.00,0.00,-32554.10,0.44,0.00,NaN,0.00,0.00,NaN
1,AUDIO,music,0.0,0.00,-2111.50,-5356.13,0.35,0.00,NaN,0.00,0.00,NaN
2,BTC,mineable,0.0,0.00,0.00,0.00,30426.90,0.00,NaN,0.00,0.00,NaN
3,CRO,medium-of-exchange,0.0,0.00,0.00,-38306.00,0.07,0.00,NaN,0.00,0.00,NaN
4,DOGE,mineable,0.0,0.00,-62.26,-22206.31,0.09,0.00,NaN,0.00,0.00,NaN
5,ONE,platform,946.0,14672.46,0.00,-109907.30,1.00,946.04,15.51,99.37,-13726.42,-93.55
6,USDC,medium-of-exchange,0.0,0.00,-9767.84,-11950.60,NaN,NaN,NaN,NaN,NaN,NaN
7,VET,logistics,2100.0,105693.00,-8876.00,-1107818.40,NaN,NaN,50.33,NaN,NaN,NaN
8,HBAR,NaN,250.0,3285.00,0.00,-37332.50,0.02,5.65,13.14,0.59,-3279.35,-99.83
9,USD,NaN,15.0,225.00,0.00,-2475.00,0.03,0.38,15.00,0.04,-224.62,-99.83


In [106]:
# # final cosmetics
# final_lastpositions = final_lastpositions.replace('-', '0')
# for c in final_lastpositions.iloc[:,4:30].columns:
#     final_lastpositions[c] = pd.to_numeric(final_lastpositions[c].str.replace('%', ''))
# final_lastpositions.info()

In [107]:
file_path = '../outputs/final_current_positions/final_current_positions_{}.csv'.format(get_now())

directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

final_lastpositions.round(2).to_csv(file_path, index=False)




In [108]:
final_lastpositions.round(2).to_csv('../outputs/final_current_positions/final_current_positions_{}.csv'.format(get_now()), index=False)

            

## Grouped DF's

In [109]:
grouped_sect = final_lastpositions.groupby(['sector']).agg(
    {'ticker': 'count', 'current_value': 'sum', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by='current_value', ascending= False).reset_index().round(2)
grouped_sect['weight'] = round(grouped_sect.current_value/grouped_sect.current_value.sum()*100, 2)
grouped_sect


,sector,ticker,current_value,cml_cost,gain_loss,weight
0,platform,1,946.04,14672.46,0.00,100.0
1,dpos,1,0.00,0.00,0.00,0.0
2,logistics,1,0.00,105693.00,-8876.00,0.0
3,medium-of-exchange,2,0.00,0.00,-9767.84,0.0
4,mineable,2,0.00,0.00,-62.26,0.0
5,music,1,0.00,0.00,-2111.50,0.0


In [110]:
file_path = '../outputs/grouped/grouped_sect_{}.csv'.format(get_now())

directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

grouped_sect.to_csv(file_path, index=False)

grouped_sect.to_csv('../outputs/grouped/grouped_sect_{}.csv'.format(get_now()), index=False)


In [111]:
grouped_sect_ind = final_lastpositions.groupby(['sector']).agg(
    {'ticker': 'count', 'current_value': 'sum', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by='current_value', ascending= False).reset_index().round(2)
grouped_sect_ind['weight'] = round(grouped_sect_ind.current_value/grouped_sect_ind.current_value.sum()*100, 2)
grouped_sect_ind


,sector,ticker,current_value,cml_cost,gain_loss,weight
0,platform,1,946.04,14672.46,0.00,100.0
1,dpos,1,0.00,0.00,0.00,0.0
2,logistics,1,0.00,105693.00,-8876.00,0.0
3,medium-of-exchange,2,0.00,0.00,-9767.84,0.0
4,mineable,2,0.00,0.00,-62.26,0.0
5,music,1,0.00,0.00,-2111.50,0.0


In [112]:
file_path = '../outputs/grouped/grouped_sect_ind_{}.csv'.format(get_now())

directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

grouped_sect_ind.to_csv(file_path, index=False)

grouped_sect_ind.to_csv('../outputs/grouped/grouped_sect_ind_{}.csv'.format(get_now()), index=False)